In [2]:
def naive(pattern, text):
    P, T = len(pattern), len(text)
    occurence = []
    
    for i in range(T - P + 1):
        j = 0 
        while j < P:
            if pattern[j] != text[j + i]: break
            j += 1
            
        if j == P: occurence.append(i)
    
    return occurence

In [3]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [4]:
genome = readGenome('019-chr1.GRCh38.excerpt.fasta')

In [13]:
alu = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [14]:
naive(alu, genome)

[56922]

In [15]:
len(genome) - len(alu) + 1

799954

In [16]:
from bm_preproc import BoyerMoore

In [17]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    align_count = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            align_count += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, align_count

In [18]:
p_bm = BoyerMoore(alu)

In [20]:
boyer_moore(alu, p_bm, genome)

([56922], 165191)

In [21]:
import bisect
class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [22]:
index = Index(genome, 8)

In [35]:
def app_match(p, t, n):
    segment_len = round(len(t) / (n+1))
    matches = set()
    for i in range(n + 1):
        start = i * segment_len
        end = min((i + 1) * segment_len, len(p))
        bm = BoyerMoore(p[start:end], alphabet='ACGT')
        curr_matches, a = boyer_moore(p[start:end], bm, t)
        for m in curr_matches:
            if m < start or m-start+len(p) > len(t): continue
                
            misses = 0
            for j in range(0, start):
                if p[j] != p[m-start+j]:
                    misses += 1
                    if misses > n:
                        break
            
            for j in range(end, len(p)):
                if p[j] != p[m-start+j]:
                    misses += 1
                    if misses > n:
                        break                    
            
            if misses <= n:
                matches.add(m-start)
        
        return list(matches)
            
            
                 
        

In [40]:
app_match('GGCGCGGTGGCTCACGCCTGTAAT', genome, 2)

[364263, 717706, 657496, 56922, 262042]

In [39]:
naive('GGCGCGGTGGCTCACGCCTGTAAT', genome)      

[56922, 262042, 364263, 657496, 717706]

In [41]:
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [44]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [45]:
p = 'TTATAT'
print(ind.query(p[0:]))

[]


In [46]:
print(ind.query(p[1:]))


[1]
